In [5]:
%pip install scipy

  Obtaining dependency information for scipy from https://files.pythonhosted.org/packages/8d/be/d324ddf6b89fd1c32fecc307f04d095ce84abb52d2e88fab29d0cd8dc7a8/scipy-1.16.0-cp311-cp311-macosx_14_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 582.9 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 3.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
from keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from Dense import Dense
from convolution import Convolutional
from activations import Sigmoid
from bce import binary_cross_entropy, binary_cross_entropy_prime
from network import train, predict
from reshape_layer import Reshape

In [2]:
def preprocess_data(x, y, limit):
    zero_index = np.where(y == 0)[0][:limit]
    one_index = np.where(y == 1)[0][:limit]
    all_indices = np.hstack((zero_index, one_index))
    all_indices = np.random.permutation(all_indices)
    x, y = x[all_indices], y[all_indices]
    x = x.reshape(len(x), 1, 28, 28)
    x = x.astype("float32") / 255
    y = to_categorical(y)
    y = y.reshape(len(y), 2, 1)
    return x, y

In [3]:
# load MNIST from server, limit to 100 images per class since we're not training on GPU
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, y_train = preprocess_data(x_train, y_train, 100)
x_test, y_test = preprocess_data(x_test, y_test, 100)

In [4]:
# neural network
network = [
    Convolutional((1, 28, 28), 3, 5),
    Sigmoid(),
    Reshape((5, 26, 26), (5 * 26 * 26, 1)),
    Dense(5 * 26 * 26, 100),
    Sigmoid(),
    Dense(100, 2),
    Sigmoid()
]

In [5]:
# train
train(
    network,
    binary_cross_entropy,
    binary_cross_entropy_prime,
    x_train,
    y_train,
    epoch=20,
    learning_rate=0.1
)

1/20, error=0.41743690790221
2/20, error=0.07286951805661471
3/20, error=0.035481558338903374
4/20, error=0.021765043722228036
5/20, error=0.01597349027994994
6/20, error=0.011858434275693103
7/20, error=0.010839631205953541
8/20, error=0.00730887453334151
9/20, error=0.006260913483749411
10/20, error=0.005213012643515581
11/20, error=0.004874282552943546
12/20, error=0.004315789023583035
13/20, error=0.003959610826790451
14/20, error=0.0036556462130096067
15/20, error=0.003394427880473225
16/20, error=0.0027827531987623605
17/20, error=0.002510037423905541
18/20, error=0.0023614936813679335
19/20, error=0.0022095588605959495
20/20, error=0.002082554354040306


In [6]:
# test
correct = 0
for x, y in zip(x_test, y_test):
    output = predict(network, x)
    if np.argmax(output) == np.argmax(y):
        correct += 1

print(correct / len(x_test))

0.985
